In [1]:
import numpy as np

In [2]:
def fit(X_Train,Y_Train):
    result={}
    classes=set(Y_Train)
    for current_class in classes:
        result[current_class]={}
        result["Total_Counter"]=len(Y_Train)
        current=(Y_Train==current_class)
        X_Train_current=X_Train[current]
        Y_Train_current=Y_Train[current]
        result[current_class]["Total_Count"]=len(Y_Train_current)
        numfeatures=X_Train_current.shape[1]
        for j in range(numfeatures):
            result[current_class][j]={}
            values=set(X_Train_current[:,j])
            result[current_class]["distinct"]=len(values)
            for current_value in values:
                result[current_class][j][current_value]=(X_Train_current[:,j]==current_value).sum()
    return result

In [3]:
def probabilty(result,x,current_class):
    prob1=np.log(result[current_class]["Total_Count"])-np.log(result["Total_Counter"])
    prob2=0
    for i in range(len(x)):
        if x[i] in result[current_class][i].keys():
            prob2+=np.log(result[current_class][i][x[i]]+1)-np.log((result[current_class]["Total_Count"]+result[current_class]["distinct"]))
        else:
            prob2+=-np.log((result[current_class]["Total_Count"]+result[current_class]["distinct"]))
    return prob1+prob2

In [4]:
def predictSingle(result,x):
    classes=result.keys() # We will also have Total_Count in thes keys and that is not a class
    maxprob=-9999
    bestclass=-1
    for current_class in classes:
        if current_class=="Total_Counter":
            continue
        prob=probabilty(result,x,current_class)
        if prob>maxprob:
            maxprob=prob
            bestclass=current_class
    return bestclass

In [5]:
def predict(result,X_Test):
    Y_Test_Pred=[]
    for x in X_Test:
        Y_Test_Pred.append(predictSingle(result,x))
    return Y_Test_Pred
        
    

In [6]:
from sklearn import datasets

In [7]:
iris=datasets.load_iris()
X=iris["data"]
Y=iris["target"]

In [8]:
from sklearn import model_selection

In [9]:
X_Train,X_Test,Y_Train,Y_Test=model_selection.train_test_split(X,Y)

In [10]:
def labelit(x):
    meanval1=x.mean()
    meanval2=(meanval1)*0.5
    meanval3=(meanval1)*1.5
    
    for i in range(len(x)):
        if(x[i]>=0 and x[i]<=meanval2):
            x[i]=0
        elif(x[i]>meanval2 and x[i]<=meanval1):
            x[i]=1
        elif(x[i]>meanval1 and x[i]<=meanval3):
            x[i]=2
        elif(x[i]>meanval3):
            x[i]=3
    return x

In [11]:
for i in range(X_Train.shape[1]):
    X_Train[:,i]=labelit(X_Train[:,i])
                                       #Labelling my data
for i in range(X_Test.shape[1]):
    X_Test[:,i]=labelit(X_Test[:,i])


In [15]:
result=fit(X_Train,Y_Train)
Y_Pred=predict(result,X_Test)
print(Y_Pred)
print(Y_Test)

[0, 0, 0, 2, 0, 2, 0, 2, 0, 1, 2, 1, 0, 0, 0, 0, 1, 2, 2, 0, 1, 2, 0, 1, 0, 1, 2, 1, 2, 2, 0, 2, 2, 1, 0, 0, 1, 1]
[0 0 0 2 0 1 0 2 0 1 1 1 0 0 0 0 1 2 2 0 1 2 0 1 0 1 2 1 2 2 0 2 2 1 0 0 1
 1]


In [16]:
from sklearn.metrics import confusion_matrix

In [18]:
confusion_matrix(Y_Test,Y_Pred) # My prediction was pretty good
                                # Out of 38 enteries i only predicted 2 as wrong

array([[16,  0,  0],
       [ 0, 10,  2],
       [ 0,  0, 10]], dtype=int64)